In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

### Создание датасета:

In [2]:
X, Y = make_classification(n_samples=1000, n_features=5, n_informative=4, n_redundant=1, n_classes=2, random_state=13)

# Добавление в датасет пустых значений - написанный скрипт работает с пустыми значениями
X[300, 2] = None
X[2, 2] = None
X[100, 2] = None
X[700, 1] = None
X[500, 1] = None

# Разбиение на тренировочную и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=True, random_state=13)

### Макрос расчета энтропийного критерия Шеннона (критерий информативности классификатора дерева решений):
$$H(X) = - \sum^{K}_{k=1}p_{k}\text{log}_{2}p_{k}.$$

$$p_{k} = \frac{1}{|X|}\sum_{i\in X}[y_{i} = k].$$

Обозначим через $p_{k}$ долю объектов класса $k$ в выборке $X$. $p_{k}$ будет характеризовать вероятность выдачи класса $k$.

In [3]:
# Расчет доли объектов, равных k в выборке Y (пример: какова доля "1" по отношению ко всей выборке)
# т.е. расчет доли объектов в выборке для каждого класса
def count_object_for_k_class(Y, k):
    p = 1/Y.shape[0] * Y[Y==k].shape[0]
    return p

In [4]:
# Расчет критерия Шеннона (энтропии)
def full_entropia(Y):
    H = 0
    for k in np.unique(Y):
        p = count_object_for_k_class(Y, k)
        H += p * np.log2(p)
    
    return -H

In [5]:
# Расчет коэффициента качества 
def quality_koef(y_left_branch, y_right_branch, current_shennon_criteria):
    
    # Доля выборки, ушедшей в левую ветку
    p = y_left_branch.shape[0] / (y_left_branch.shape[0] + y_right_branch.shape[0])
    
    # Коэффициент качества
    return current_shennon_criteria - p * full_entropia(y_left_branch) - (1-p) * full_entropia(y_right_branch)

### Класс узла дерева:

In [6]:
# Класс узла дерева
class Node:
    
    def __init__(self, index, t, left_branch, right_branch):
        self.index = index        # index - индекс столбца массива признаков X, по которому производится сравнение
        self.t = t        # t - значение граничного параметра для столбца с данным индексом, по которому разделяется выборка на левую и правую части
        self.left_branch = left_branch        # left_branch - левая ветка дерева
        self.right_branch = right_branch        # right_branch - правая ветка дерева

### Класс листа дерева:

In [7]:
class Leaf:
                                    # Конструктор класса. На вход принимает массив X (признаки) и массив Y (целевые классы)
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.predicted_class = self.predict()
        
                                    # Функция - предсказывание. Предсказывает целевой класс для данного листа
                                    # Класс, объектов которого будет большинство в данном листе, будет возвращен
    def predict(self):
        count_unique_values = []
        unique_values = np.unique(self.y)
        for k in unique_values:
            count_unique_values.append(self.y[self.y==k].shape[0])
        
        maximum_count = 0
        target_class = 0
        for i, value in enumerate(count_unique_values):
            if maximum_count < value:
                target_class = unique_values[i]
                
        return target_class

### Макрос разбиения датасета в узле на левую и правую ветки:

In [8]:
# Функция разделяет выборку на две ветки: левую и правую.
# Критерий - сравнение наблюдения в массиве x по индексу index с номинальным t
# Функция обрабатывает Null - значения из массива x
def split_branch(x, y, index, t):
    
    x_left_branch = []
    y_left_branch = []
    x_right_branch = []
    y_right_branch = []
    
    for i in range(x.shape[0]):
        
        # Обработка Null - значений:
        if x[i, index] == None:
            x_left_branch.append(x[i, :])
            y_left_branch.append(y[i])
            x_right_branch.append(x[i, :])
            y_right_branch.append(y[i])
            
        elif x[i, index] <= t:
            x_left_branch.append(x[i, :])
            y_left_branch.append(y[i])
            
        else:
            x_right_branch.append(x[i, :])
            y_right_branch.append(y[i])
    
    return np.array(x_left_branch), np.array(y_left_branch), np.array(x_right_branch), np.array(y_right_branch)

### Макрос поиска наилучшего критерия для разбиения на левую и правую ветки:

In [9]:
def find_best_split(x, y, min_count_of_samples_per_branch = 5):

# Критерий выбора наилучшего значения: максимальное значение параметра качества - best_quality (прирост информации)
    best_t = None    # Значение параметра t, при котором достигается наилучшее качество (прирост информации)
    best_index = None    # Значение индекса столбца  массива x, при котором достигается наилучшее качество (прирост информации)  
    best_quality = 0    # Переменная, хранящая значения качества (прирост информации)
    current_entropia = full_entropia(y)    # Переменная, хранящая текущую энтропию для данной выборки
    
    # Перебор всех значений x по всем строкам и столбцам для поиска индекса и параметра t с наибольшим приростом информации
    for index in range(x.shape[1]):
        for row in range(x.shape[0]):
            t = x[row,index]
            x_left_branch, y_left_branch, x_right_branch, y_right_branch = split_branch(x, y, index, t)
            
            # Проверка, достаточно ли образцов и листе - борьба с переобучением
            if y_left_branch.shape[0] < min_count_of_samples_per_branch or y_right_branch.shape[0] < min_count_of_samples_per_branch:
                continue
            
            # Расчет прироста информации на данной итерации
            current_quality = quality_koef(y_left_branch, y_right_branch, current_entropia)
            
            if current_quality > best_quality:
                best_quality = current_quality
                best_index = index
                best_t = x[row,index]
                
    return best_index, best_t, best_quality       

### Cоздание дерева решений классификатора

In [10]:
def build_tree(x, y, min_count_of_samples_per_branch = 5, max_depth=100):
    best_index, best_t, best_quality = find_best_split(x, y, min_count_of_samples_per_branch=min_count_of_samples_per_branch)
    
    #  Базовый случай — прекращаем рекурсию, когда нет прироста в качества или когда образцов в листе меньше заданного
    if best_quality == 0 or y.shape[0] < min_count_of_samples_per_branch:
        return Leaf(x, y)
    
    x_left_branch, y_left_branch, x_right_branch, y_right_branch = split_branch(x, y, best_index, best_t)
    
    # Рекурсивное добавление левой и правой веток
    left_branch = build_tree(x_left_branch, y_left_branch, min_count_of_samples_per_branch=min_count_of_samples_per_branch, max_depth=max_depth)
    right_branch = build_tree(x_right_branch, y_right_branch, min_count_of_samples_per_branch=min_count_of_samples_per_branch, max_depth=max_depth)
    
    return Node(best_index, best_t, left_branch, right_branch)    

### Функция предсказывания деревом:

In [11]:
def classify_object(x, tree_subclass):
    # Функция предсказывания для одного наблюдения из массива X
    #  Останавливаем рекурсию, если достигли класса листа - именно листья возвращают нам наш целевой класс
    if isinstance(tree_subclass, Leaf):
        answer = tree_subclass.predict()
        return answer
    
    #  Сравнение значений в массиве x с целевыми параметрами t
    if x[tree_subclass.index] <= tree_subclass.t:
        return classify_object(x, tree_subclass.left_branch)
    else:
        return classify_object(x, tree_subclass.right_branch)

In [12]:
def tree_predict(x, tree):
    # Функция предсказывания значений для всех наблюдений в массиве X
    classes = []
    for sample in x:
        prediction = classify_object(sample, tree)
        classes.append(prediction)
    return classes

### Метод оценки точности работы классификатора - расчета accuracy:

In [13]:
def get_accuracy(y_true, y_pred):
    k = 0
    for i in range(y_true.shape[0]):
        if y_true[i] == y_pred[i]:
            k += 1
    return k / y_true.shape[0]

### Запуск и работа скрипта представлены ниже:

In [14]:
# Создание и обучение дерева решений
tree = build_tree(x_train, y_train, min_count_of_samples_per_branch=3)

In [15]:
# Предсказание на тренировочной выборке
y_pred_train = tree_predict(x_train, tree)
print(f'Точность предсказания на обучающей выборке составляет {round(get_accuracy(y_train, y_pred_train), 3)}')

Точность предсказания на обучающей выборке составляет 0.957


In [16]:
# Предсказание на тестовой выборке
y_test_pred = tree_predict(x_test, tree)
print(f'Точность предсказания на обучающей выборке составляет {round(get_accuracy(y_test, y_test_pred), 3)}')

Точность предсказания на обучающей выборке составляет 0.89
